In [2]:
# import data manipulation libraries
import numpy as np
import pandas as pd
# import file acquisition tools
import os
import requests
# import splitting function
from sklearn.model_selection import train_test_split

In [1]:
def acquire_austin_animal_shelter_data():
    '''
    This function will read in austin animal shelter intake and outcome data via csv,
    it will then rename to the column names to lowercase and remove spaces,
    it will then combine the csv files into one DataFrame and return the combined df.
    '''
    # set the filenames we are looking for
    intake_filename = 'Austin_Animal_Center_Intakes.csv'
    outcome_filename = 'Austin_Animal_Center_Outcomes.csv'
    # set the api paths
    intake_api = 'https://data.austintexas.gov/resource/wter-evkm.json'
    outcome_api = 'https://data.austintexas.gov/resource/9t4d-g238.json'
    
    # check if intake and outcome files exist in the local directory
    if os.path.exists(intake_filename):
        # read in csv files for intakes for animals
        intakes = pd.read_csv(intake_filename)
    else:
        # if the intake file does not exist locally, then download the data via api
        intakes = pd.DataFrame(request.json())
        intakes.datetime = pd.to_datetime(intakes.datetime)
    
    # check if outcome file exist in the local directory
    if os.path.exists(outcome_filename):
        # read in csv files for outcomes for animals
        outcomes = pd.read_csv(outcome_filename)
    else:
        # if the intake file does not exist locally, then download the data via api
        outcomes = pd.DataFrame(request.json())
        outcomes.datetime = pd.to_datetime(outcomes.datetime)
        
    # rename column names to lowercase and remove spaces
    intakes.columns = intakes.columns.str.lower().str.replace(' ','_').to_list()
    outcomes.columns = outcomes.columns.str.lower().str.replace(' ','_').to_list()
    # merge dataframes into one
    animals = pd.merge(left=intakes, right=outcomes, how='inner', 
                       on='animal_id', suffixes=('_in','_out'))
    
    # return the combined dataframe
    return animals

In [2]:
def prepare_austin_animal_shelter(animals):
    '''
    this function will prepare the animal shelter data by changing the datetime_in and out
    into datatime dtype, it will then drop the redundant color_out, breed_out, name_out
    and animal_type_out columns and rename the remaining columns, it will then
    return the modified dataframe.
    '''
    # change dtypes to datetime
    animals['datetime_in'] = pd.to_datetime(animals.datetime_in)
    animals['datetime_out'] = pd.to_datetime(animals.datetime_out)
    # remove redundant columns
    animals = animals.drop(columns=['color_out', 'breed_out', 
                                    'name_out', 'animal_type_out'])
    # rename remaining columns
    animals = animals.rename(columns={'color_in':'color', 
                                      'breed_in':'breed', 'name_in':'name',
                                      'animal_type_in':'animal_type'})
    # drop redundaant columns for month_year
#     animals = animals.drop(columns=['monthyear_in', 'monthyear_out'])
    # remove the 2 rows with nullsin the sex_upon_intake column, 
    # 1 of which is a test row
    animals = animals[animals.sex_upon_intake.isna() == False]
    
    # create a new column with binned outcome_type
    animals['outcome'] = np.where(animals.outcome_type == 'Adoption', 'adopted',
                          np.where(animals.outcome_type == 'Return to Owner', 'returned',
                          np.where(animals.outcome_type == 'Transfer', 'transfered',
                          np.where(animals.outcome_type == 'Euthanasia', 'death',
                          np.where(animals.outcome_type == 'Rto-Adopt', 'returned',
                          np.where(animals.outcome_type == 'Died', 'death',
                          np.where(animals.outcome_type == 'Disposal', 'death',
                          np.where(animals.outcome_type == 'Missing', 'unknown',
                          np.where(animals.outcome_type == 'Stolen', 'unknown',
                          np.where(animals.outcome_type == 'Relocate', 'transfered',
                          np.where(animals.outcome_type.isnull() == True, 'unknown', ''
                                  )))))))))))
    # return the modified dataframe
    return animals

In [3]:
def split_austin_animal_shelter(df):
    '''
    This function splits a dataframe into 
    train, validate, and test in order to explore the data and to create and validate models. 
    It takes in a dataframe and contains an integer for setting a seed for replication. 
    Test is 20% of the original dataset. The remaining 80% of the dataset is 
    divided between valiidate and train, with validate being .30*.80= 24% of 
    the original dataset, and train being .70*.80= 56% of the original dataset. 
    The function returns, train, validate and test dataframes. 
    '''
    # Here we are spliting train into .8 of the original dataset. 
    # and test into 20% of the original dataset.
    train, test = train_test_split(df, test_size = .2, random_state=123)
    # here we assign validate to be .3 of the train dataset 
    train, validate = train_test_split(train, test_size=.3, random_state=123)
    # returns train validate and test dataframes
    return train, validate, test

In [5]:
def wrangle_austin_animal_shelter():
    '''
    This function will perform acquisition, preparation and train, validate, test
    split into one function call and will return the cleaned dataframe along with 
    the train, validate and test dataframes.
    '''
    # check if a cached file of the dataset exists in the local directory
    
    # acquire data from csv files
    animals = acquire_austin_animal_shelter_data()
    # prepare the data
    animals = prepare_austin_animal_shelter(animals)
    # split data into train, validate and test groups
    train, validate, test = split_austin_animal_shelter(animals)
    # return all the data
    return animals, train, validate, test

In [2]:
intakes = pd.read_csv('Austin_Animal_Center_Intakes.csv')
outcomes = pd.read_csv('Austin_Animal_Center_Outcomes.csv')

In [4]:
intake_api = 'https://data.austintexas.gov/resource/wter-evkm.json'
outcome_api = 'https://data.austintexas.gov/resource/9t4d-g238.json'

In [6]:
requests.get(intake_api)

<Response [200]>

In [7]:
request = requests.get(intake_api)
request.json()

[{'animal_id': 'A665644',
  'datetime': '2013-10-21T07:59:00.000',
  'datetime2': '2013-10-21T07:59:00.000',
  'found_location': 'Austin (TX)',
  'intake_type': 'Stray',
  'intake_condition': 'Sick',
  'animal_type': 'Cat',
  'sex_upon_intake': 'Intact Female',
  'age_upon_intake': '4 weeks',
  'breed': 'Domestic Shorthair Mix',
  'color': 'Calico'},
 {'animal_id': 'A665739',
  'name': '*Alana',
  'datetime': '2013-10-22T11:11:00.000',
  'datetime2': '2013-10-22T11:11:00.000',
  'found_location': 'Austin (TX)',
  'intake_type': 'Stray',
  'intake_condition': 'Normal',
  'animal_type': 'Cat',
  'sex_upon_intake': 'Intact Female',
  'age_upon_intake': '1 month',
  'breed': 'Domestic Medium Hair Mix',
  'color': 'Black'},
 {'animal_id': 'A665763',
  'datetime': '2013-10-22T15:10:00.000',
  'datetime2': '2013-10-22T15:10:00.000',
  'found_location': 'E Riverside Dr/Royal Crest Dr in Austin (TX)',
  'intake_type': 'Stray',
  'intake_condition': 'Normal',
  'animal_type': 'Dog',
  'sex_upon_

In [28]:
requests.get('https://data.austintexas.gov/resource/wter-evkm?$offset=200000').json()

[]

In [35]:
url= 'https://data.austintexas.gov/resource/wter-evkm'
print(url)

https://data.austintexas.gov/resource/wter-evkm


In [33]:
x=1000

In [36]:
url = url + f'?$offset={x}'
print(url)

https://data.austintexas.gov/resource/wter-evkm?$offset=1000


In [ ]:
def get_data(url, offset=1000):
    '''
    this function will return a dataFrame with all of the results from the passed url
    until the last page of results
    '''
    # create request.get for passed url
    page = requests.get(url)
    # create a dataframe withe the first page results
    df = pd.DataFrame(page.json())
    # set first offset amount
    x=offset
    # loop through all pages of results for passed url
    while page.json():
        # set the page url to the next page url
        next_url = url + f'?$offset={x}'
        page = requests.get(next_url)
        df = pd.concat([df, pd.DataFrame(page.json())], axis=0)
    # return the datframe
    return df.reset_index().drop(columns='index')

In [8]:
pd.DataFrame(request.json())

,animal_id,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name
0,A665644,2013-10-21T07:59:00.000,2013-10-21T07:59:00.000,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN
1,A665739,2013-10-22T11:11:00.000,2013-10-22T11:11:00.000,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,Black,*Alana
2,A665763,2013-10-22T15:10:00.000,2013-10-22T15:10:00.000,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,Tan/White,NaN
3,A379998,2013-10-23T11:42:00.000,2013-10-23T11:42:00.000,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,Black,Disciple
4,A634503,2013-10-01T14:49:00.000,2013-10-01T14:49:00.000,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan,Otter
...,...,...,...,...,...,...,...,...,...,...,...,...
995,A664268,2013-10-01T12:05:00.000,2013-10-01T12:05:00.000,Austin (TX),Stray,Normal,Cat,Intact Female,3 years,Russian Blue Mix,Blue,*Kochka
996,A656696,2013-10-13T12:52:00.000,2013-10-13T12:52:00.000,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Brown Tabby,*Angel
997,A665285,2013-10-15T18:19:00.000,2013-10-15T18:19:00.000,4807 Airport Blvd in Austin (TX),Stray,Normal,Cat,Intact Female,5 months,Domestic Shorthair Mix,Torbie,Tabby
998,A665185,2013-10-14T18:32:00.000,2013-10-14T18:32:00.000,Hwy 183/Burnet Rd in Austin (TX),Stray,Normal,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Brown Tabby/White,*Vince


In [9]:
intake_a = pd.DataFrame(request.json())

In [10]:
intake_a.datetime = pd.to_datetime(intake_a.datetime)

In [11]:
intake_a.head()

,animal_id,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name
0,A665644,2013-10-21 07:59:00,2013-10-21T07:59:00.000,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN
1,A665739,2013-10-22 11:11:00,2013-10-22T11:11:00.000,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,Black,*Alana
2,A665763,2013-10-22 15:10:00,2013-10-22T15:10:00.000,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,Tan/White,NaN
3,A379998,2013-10-23 11:42:00,2013-10-23T11:42:00.000,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,Black,Disciple
4,A634503,2013-10-01 14:49:00,2013-10-01T14:49:00.000,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan,Otter


In [12]:
intake_a.datetime2 = pd.to_datetime(intake_a.datetime2)

In [20]:
pd.concat([intake_a.datetime2.dt.month_name(), intake_a.datetime2.dt.year], axis=1)

,datetime2,datetime2
0,October,2013
1,October,2013
2,October,2013
3,October,2013
4,October,2013
...,...,...
995,October,2013
996,October,2013
997,October,2013
998,October,2013


In [3]:
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
1,A665739,*Alana,10/22/2013 11:11:00 AM,October 2013,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,Black
2,A665763,NaN,10/22/2013 03:10:00 PM,October 2013,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,Tan/White
3,A379998,Disciple,10/23/2013 11:42:00 AM,October 2013,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,Black
4,A634503,Otter,10/01/2013 02:49:00 PM,October 2013,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan


In [4]:
intakes.columns = intakes.columns.str.lower().str.replace(' ','_').to_list()
outcomes.columns = outcomes.columns.str.lower().str.replace(' ','_').to_list()

In [5]:
intakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150370 entries, 0 to 150369
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         150370 non-null  object
 1   name              106773 non-null  object
 2   datetime          150370 non-null  object
 3   monthyear         150370 non-null  object
 4   found_location    150370 non-null  object
 5   intake_type       150370 non-null  object
 6   intake_condition  150370 non-null  object
 7   animal_type       150370 non-null  object
 8   sex_upon_intake   150368 non-null  object
 9   age_upon_intake   150369 non-null  object
 10  breed             150370 non-null  object
 11  color             150370 non-null  object
dtypes: object(12)
memory usage: 13.8+ MB


In [6]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150395 entries, 0 to 150394
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         150395 non-null  object
 1   name              106910 non-null  object
 2   datetime          150395 non-null  object
 3   monthyear         150395 non-null  object
 4   date_of_birth     150395 non-null  object
 5   outcome_type      150370 non-null  object
 6   outcome_subtype   68904 non-null   object
 7   animal_type       150395 non-null  object
 8   sex_upon_outcome  150393 non-null  object
 9   age_upon_outcome  150348 non-null  object
 10  breed             150395 non-null  object
 11  color             150395 non-null  object
dtypes: object(12)
memory usage: 13.8+ MB


In [38]:
animals = pd.merge(left=intakes, right=outcomes, how='inner', on='animal_id',
                   suffixes=('_in','_out'))

In [12]:
animals.head()

,animal_id,name_in,datetime_in,monthyear_in,found_location,intake_type,intake_condition,animal_type_in,sex_upon_intake,age_upon_intake,...,datetime_out,monthyear_out,date_of_birth,outcome_type,outcome_subtype,animal_type_out,sex_upon_outcome,age_upon_outcome,breed_out,color_out
0,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,...,10/21/2013 11:39:00 AM,Oct 2013,09/21/2013,Transfer,Partner,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
1,A665739,*Alana,10/22/2013 11:11:00 AM,October 2013,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,...,12/20/2013 05:36:00 PM,Dec 2013,09/16/2013,Adoption,Foster,Cat,Spayed Female,3 months,Domestic Medium Hair Mix,Black
2,A665763,NaN,10/22/2013 03:10:00 PM,October 2013,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,...,10/26/2013 06:32:00 PM,Oct 2013,06/22/2013,Adoption,NaN,Dog,Neutered Male,4 months,Cairn Terrier Mix,Tan/White
3,A379998,Disciple,10/23/2013 11:42:00 AM,October 2013,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,...,10/25/2013 12:53:00 PM,Oct 2013,02/21/2003,Return to Owner,NaN,Dog,Intact Male,10 years,Pit Bull,Black
4,A634503,Otter,10/01/2013 02:49:00 PM,October 2013,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,...,10/02/2013 12:40:00 PM,Oct 2013,08/11/2011,Adoption,NaN,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan


In [20]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193118 entries, 0 to 193117
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         193118 non-null  object
 1   name_in           148817 non-null  object
 2   datetime_in       193118 non-null  object
 3   monthyear_in      193118 non-null  object
 4   found_location    193118 non-null  object
 5   intake_type       193118 non-null  object
 6   intake_condition  193118 non-null  object
 7   animal_type_in    193118 non-null  object
 8   sex_upon_intake   193116 non-null  object
 9   age_upon_intake   193117 non-null  object
 10  breed_in          193118 non-null  object
 11  color_in          193118 non-null  object
 12  name_out          148817 non-null  object
 13  datetime_out      193118 non-null  object
 14  monthyear_out     193118 non-null  object
 15  date_of_birth     193118 non-null  object
 16  outcome_type      193082 non-null  obj

In [15]:
animals.name_in == animals.name_out

0         False
1          True
2         False
3          True
4          True
          ...  
193113     True
193114     True
193115     True
193116     True
193117     True
Length: 193118, dtype: bool

In [17]:
animals = animals.drop(columns=['monthyear_in', 'monthyear_out'])

In [18]:
animals.head()

,animal_id,name_in,datetime_in,found_location,intake_type,intake_condition,animal_type_in,sex_upon_intake,age_upon_intake,breed_in,...,name_out,datetime_out,date_of_birth,outcome_type,outcome_subtype,animal_type_out,sex_upon_outcome,age_upon_outcome,breed_out,color_out
0,A665644,NaN,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,...,NaN,10/21/2013 11:39:00 AM,09/21/2013,Transfer,Partner,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
1,A665739,*Alana,10/22/2013 11:11:00 AM,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,...,*Alana,12/20/2013 05:36:00 PM,09/16/2013,Adoption,Foster,Cat,Spayed Female,3 months,Domestic Medium Hair Mix,Black
2,A665763,NaN,10/22/2013 03:10:00 PM,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,...,NaN,10/26/2013 06:32:00 PM,06/22/2013,Adoption,NaN,Dog,Neutered Male,4 months,Cairn Terrier Mix,Tan/White
3,A379998,Disciple,10/23/2013 11:42:00 AM,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,...,Disciple,10/25/2013 12:53:00 PM,02/21/2003,Return to Owner,NaN,Dog,Intact Male,10 years,Pit Bull,Black
4,A634503,Otter,10/01/2013 02:49:00 PM,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,...,Otter,10/02/2013 12:40:00 PM,08/11/2011,Adoption,NaN,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan


In [22]:
animals.shape

(193118, 23)

In [23]:
animals.animal_id.nunique()

133921

In [24]:
animals.animal_id.value_counts().sort_values()

A717160       1
A732194       1
A844725       1
A741841       1
A737046       1
           ... 
A700407      81
A706536     121
A718877     144
A718223     196
A721033    1089
Name: animal_id, Length: 133921, dtype: int64

In [49]:
# repeats = animals[animals.animal_id.value_counts().sort_values() > 1]

In [34]:
len(intakes[intakes.animal_id == 'A721033'])

33

In [35]:
len(outcomes[outcomes.animal_id == 'A721033'])

33

In [39]:
intakes[intakes.animal_id == 'A721033']

,animal_id,name,datetime,monthyear,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
6601,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
14155,A721033,Lil Bit,03/07/2018 08:27:00 AM,March 2018,4111 South 1St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,Rat Terrier Mix,Tricolor/Brown Brindle
16598,A721033,Lil Bit,05/20/2019 10:18:00 AM,May 2019,1819 South Pleasant Valley Road in Austin (TX),Public Assist,Normal,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle
17205,A721033,Lil Bit,04/05/2018 05:59:00 PM,April 2018,9705 Manchaca Road in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,Rat Terrier Mix,Tricolor/Brown Brindle
25150,A721033,Lil Bit,09/24/2018 07:44:00 PM,September 2018,Shady And Airport in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
29646,A721033,Lil Bit,03/15/2017 09:24:00 AM,March 2017,E Ben White Blvd & Montopolis Dr in Austin (TX),Public Assist,Normal,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
31643,A721033,Lil Bit,02/12/2019 10:21:00 AM,February 2019,1936 East Oltorf Street in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
32254,A721033,Lil Bit,10/20/2016 10:47:00 PM,October 2016,6200 E. Ben White Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
33464,A721033,Lil Bit,09/21/2018 03:01:00 PM,September 2018,863 Airport Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
33858,A721033,Lil Bit,10/05/2019 12:31:00 PM,October 2019,1135 Airport Boulevard in Austin (TX),Public Assist,Normal,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle


In [36]:
outcomes[outcomes.animal_id == 'A721033']

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
4811,A721033,Lil Bit,08/10/2019 11:56:00 AM,Aug 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle
10690,A721033,Lil Bit,01/10/2017 04:20:00 PM,Jan 2017,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
13474,A721033,Lil Bit,10/21/2016 06:55:00 PM,Oct 2016,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
16272,A721033,Lil Bit,03/11/2019 04:27:00 PM,Mar 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
19171,A721033,Lil Bit,05/21/2019 02:42:00 PM,May 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle
23727,A721033,Lil Bit,11/24/2018 04:25:00 PM,Nov 2018,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
26653,A721033,Lil Bit,03/16/2018 04:04:00 PM,Mar 2018,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,2 years,Rat Terrier Mix,Tricolor/Brown Brindle
30401,A721033,Lil Bit,05/04/2019 08:37:00 AM,May 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
35824,A721033,Lil Bit,02/18/2019 04:46:00 PM,Feb 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
40416,A721033,Lil Bit,01/28/2017 03:22:00 PM,Jan 2017,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle


In [27]:
lil_bit = animals[animals.animal_id == 'A721033']

In [30]:
lil_bit

,animal_id,name_in,datetime_in,monthyear_in,found_location,intake_type,intake_condition,animal_type_in,sex_upon_intake,age_upon_intake,...,datetime_out,monthyear_out,date_of_birth,outcome_type,outcome_subtype,animal_type_out,sex_upon_outcome,age_upon_outcome,breed_out,color_out
12649,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,...,08/10/2019 11:56:00 AM,Aug 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle
12650,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,...,01/10/2017 04:20:00 PM,Jan 2017,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
12651,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,...,10/21/2016 06:55:00 PM,Oct 2016,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
12652,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,...,03/11/2019 04:27:00 PM,Mar 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle
12653,A721033,Lil Bit,02/24/2019 09:53:00 PM,February 2019,700 Allen St in Austin (TX),Public Assist,Normal,Dog,Neutered Male,3 years,...,05/21/2019 02:42:00 PM,May 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13733,A721033,Lil Bit,02/22/2018 10:28:00 AM,February 2018,6400 Ben White Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,...,09/12/2016 01:40:00 PM,Sep 2016,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Rat Terrier Mix,Tricolor/Brown Brindle
13734,A721033,Lil Bit,02/22/2018 10:28:00 AM,February 2018,6400 Ben White Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,...,03/08/2018 03:04:00 PM,Mar 2018,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,2 years,Rat Terrier Mix,Tricolor/Brown Brindle
13735,A721033,Lil Bit,02/22/2018 10:28:00 AM,February 2018,6400 Ben White Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,...,04/17/2018 11:07:00 AM,Apr 2018,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,2 years,Rat Terrier Mix,Tricolor/Brown Brindle
13736,A721033,Lil Bit,02/22/2018 10:28:00 AM,February 2018,6400 Ben White Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,...,02/12/2019 03:20:00 PM,Feb 2019,05/20/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rat Terrier Mix,Tricolor/Brown Brindle


In [28]:
lil_bit.datetime_in.value_counts()

02/24/2019 09:53:00 PM    33
04/26/2019 10:28:00 AM    33
08/18/2018 09:45:00 PM    33
08/06/2018 07:00:00 PM    33
01/26/2017 06:55:00 AM    33
09/03/2016 02:30:00 PM    33
01/09/2017 02:26:00 PM    33
05/22/2018 01:48:00 PM    33
09/16/2019 02:06:00 PM    33
02/06/2017 10:13:00 AM    33
02/20/2016 10:44:00 AM    33
11/19/2018 07:53:00 PM    33
04/22/2018 02:40:00 PM    33
12/17/2018 04:52:00 PM    33
02/16/2019 10:30:00 AM    33
04/12/2018 10:14:00 AM    33
03/14/2018 10:57:00 AM    33
03/07/2018 08:27:00 AM    33
01/30/2017 11:05:00 PM    33
03/07/2019 12:02:00 PM    33
09/03/2018 05:24:00 PM    33
12/15/2016 10:07:00 AM    33
08/05/2019 10:52:00 AM    33
07/10/2016 11:53:00 AM    33
10/05/2019 12:31:00 PM    33
09/21/2018 03:01:00 PM    33
10/20/2016 10:47:00 PM    33
02/12/2019 10:21:00 AM    33
03/15/2017 09:24:00 AM    33
09/24/2018 07:44:00 PM    33
04/05/2018 05:59:00 PM    33
05/20/2019 10:18:00 AM    33
02/22/2018 10:28:00 AM    33
Name: datetime_in, dtype: int64

In [29]:
lil_bit.datetime_out.value_counts()

08/10/2019 11:56:00 AM    33
08/09/2018 11:13:00 AM    33
02/12/2019 03:20:00 PM    33
04/17/2018 11:07:00 AM    33
03/08/2018 03:04:00 PM    33
09/12/2016 01:40:00 PM    33
12/16/2016 11:32:00 AM    33
03/16/2017 12:50:00 PM    33
12/27/2018 03:41:00 PM    33
02/23/2018 01:06:00 PM    33
09/08/2018 01:55:00 PM    33
09/23/2018 02:22:00 PM    33
02/02/2017 11:19:00 AM    33
02/20/2016 04:18:00 PM    33
10/20/2019 11:35:00 AM    33
07/11/2016 05:44:00 PM    33
09/19/2019 11:02:00 AM    33
01/10/2017 04:20:00 PM    33
02/07/2017 05:26:00 PM    33
04/11/2018 12:37:00 PM    33
09/25/2018 06:14:00 PM    33
02/26/2019 07:00:00 PM    33
04/24/2018 04:12:00 PM    33
08/22/2018 01:21:00 PM    33
01/28/2017 03:22:00 PM    33
02/18/2019 04:46:00 PM    33
05/04/2019 08:37:00 AM    33
03/16/2018 04:04:00 PM    33
11/24/2018 04:25:00 PM    33
05/21/2019 02:42:00 PM    33
03/11/2019 04:27:00 PM    33
10/21/2016 06:55:00 PM    33
05/26/2018 03:52:00 PM    33
Name: datetime_out, dtype: int64

In [31]:
lil_bit.found_location.value_counts()

6210 E Ben White in Austin (TX)                           66
700 Allen St in Austin (TX)                               33
South 1St And Stassney in Austin (TX)                     33
1500 Red River Street in Austin (TX)                      33
606 West Slaughter in Austin (TX)                         33
901 W Ben White Blvd in Austin (TX)                       33
6010 E Riverside in Austin (TX)                           33
6210 E Ben White Blvd in Austin (TX)                      33
W Oltorf And S 1St St in Austin (TX)                      33
Justin Lane And North Lamar in Austin (TX)                33
2508 E Riverside Dr in Austin (TX)                        33
9300 S Ih 35 in Austin (TX)                               33
South First Street And Oltorf Street in Austin (TX)       33
1144 Airport in Austin (TX)                               33
1135 Airport Blvd in Austin (TX)                          33
Austin (TX)                                               33
814 Stark Street in Aust

In [32]:
lil_bit.outcome_type.value_counts()

Return to Owner    1023
Rto-Adopt            66
Name: outcome_type, dtype: int64

### Every intake time is being linked to every outcome time, so the amount of rows for return animals are squared of actual

In [51]:
(animals.color_in != animals.color_out).sum()

0

### color in is the same as color out for all animals

In [52]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193118 entries, 0 to 193117
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         193118 non-null  object
 1   name_in           148817 non-null  object
 2   datetime_in       193118 non-null  object
 3   monthyear_in      193118 non-null  object
 4   found_location    193118 non-null  object
 5   intake_type       193118 non-null  object
 6   intake_condition  193118 non-null  object
 7   animal_type_in    193118 non-null  object
 8   sex_upon_intake   193116 non-null  object
 9   age_upon_intake   193117 non-null  object
 10  breed_in          193118 non-null  object
 11  color_in          193118 non-null  object
 12  name_out          148817 non-null  object
 13  datetime_out      193118 non-null  object
 14  monthyear_out     193118 non-null  object
 15  date_of_birth     193118 non-null  object
 16  outcome_type      193082 non-null  obj

In [53]:
(animals.breed_in != animals.breed_out).sum()

0

#### Breed_in is the same as breed_out

In [62]:
(animals.name_in.isna() == animals.name_out.isna()).sum() / 193118

1.0

In [63]:
(~animals.name_in.isna() == ~animals.name_out.isna()).sum() / 193118

1.0

#### name_in is the same as name_out

In [65]:
animals = animals.drop(columns=['name_out'])
animals = animals.rename(columns={'name_in':'name'})

In [71]:
(animals.animal_type_in != animals.animal_type_out).sum()

0

#### animal_type_in is the same as animal_type_out

In [76]:
animals = animals.drop(columns=['animal_type_out'])
animals = animals.rename(columns={'animal_type_in':'animal_type'})

KeyError: "['animal_type_out'] not found in axis"

In [64]:
animals = animals.drop(columns=['color_out', 'breed_out', 
                                'name_out', 'animal_type_out'])
animals = animals.rename(columns={'color_in':'color', 
                                  'breed_in':'breed', 'name_in':'name',
                                  'animal_type_in':'animal_type'})


KeyError: "['color_out', 'breed_out'] not found in axis"

In [73]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193118 entries, 0 to 193117
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   animal_id         193118 non-null  object        
 1   name              148817 non-null  object        
 2   datetime_in       193118 non-null  datetime64[ns]
 3   monthyear_in      193118 non-null  object        
 4   found_location    193118 non-null  object        
 5   intake_type       193118 non-null  object        
 6   intake_condition  193118 non-null  object        
 7   animal_type       193118 non-null  object        
 8   sex_upon_intake   193116 non-null  object        
 9   age_upon_intake   193117 non-null  object        
 10  breed             193118 non-null  object        
 11  color             193118 non-null  object        
 12  datetime_out      193118 non-null  datetime64[ns]
 13  monthyear_out     193118 non-null  object        
 14  date

In [67]:
animals['datetime_in'] = pd.to_datetime(animals.datetime_in)
animals['datetime_out'] = pd.to_datetime(animals.datetime_out)

In [68]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193118 entries, 0 to 193117
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   animal_id         193118 non-null  object        
 1   name              148817 non-null  object        
 2   datetime_in       193118 non-null  datetime64[ns]
 3   monthyear_in      193118 non-null  object        
 4   found_location    193118 non-null  object        
 5   intake_type       193118 non-null  object        
 6   intake_condition  193118 non-null  object        
 7   animal_type_in    193118 non-null  object        
 8   sex_upon_intake   193116 non-null  object        
 9   age_upon_intake   193117 non-null  object        
 10  breed             193118 non-null  object        
 11  color             193118 non-null  object        
 12  datetime_out      193118 non-null  datetime64[ns]
 13  monthyear_out     193118 non-null  object        
 14  date

In [69]:
animals.head()

,animal_id,name,datetime_in,monthyear_in,found_location,intake_type,intake_condition,animal_type_in,sex_upon_intake,age_upon_intake,breed,color,datetime_out,monthyear_out,date_of_birth,outcome_type,outcome_subtype,animal_type_out,sex_upon_outcome,age_upon_outcome
0,A665644,NaN,2013-10-21 07:59:00,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,2013-10-21 11:39:00,Oct 2013,09/21/2013,Transfer,Partner,Cat,Intact Female,4 weeks
1,A665739,*Alana,2013-10-22 11:11:00,October 2013,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,Black,2013-12-20 17:36:00,Dec 2013,09/16/2013,Adoption,Foster,Cat,Spayed Female,3 months
2,A665763,NaN,2013-10-22 15:10:00,October 2013,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,Tan/White,2013-10-26 18:32:00,Oct 2013,06/22/2013,Adoption,NaN,Dog,Neutered Male,4 months
3,A379998,Disciple,2013-10-23 11:42:00,October 2013,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,Black,2013-10-25 12:53:00,Oct 2013,02/21/2003,Return to Owner,NaN,Dog,Intact Male,10 years
4,A634503,Otter,2013-10-01 14:49:00,October 2013,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan,2013-10-02 12:40:00,Oct 2013,08/11/2011,Adoption,NaN,Dog,Spayed Female,2 years


In [74]:
animals.outcome_subtype.value_counts()

Partner                39032
Foster                 16298
Rabies Risk             4383
Suffering               3825
Snr                     3528
SCRP                    3507
Out State               1057
Aggressive               763
In Kennel                758
Offsite                  626
In Foster                364
Medical                  350
At Vet                   337
Field                    267
Behavior                 186
Enroute                   97
Court/Investigation       94
Underage                  37
Possible Theft            34
Customer S                30
In Surgery                30
Prc                       15
Barn                      14
Emergency                 14
In State                  12
Emer                       7
Name: outcome_subtype, dtype: int64

In [75]:
animals.outcome_type.value_counts()

Adoption           91428
Transfer           47153
Return to Owner    40231
Euthanasia         10118
Rto-Adopt           1866
Died                1448
Disposal             698
Missing              103
Relocate              26
Stolen                11
Name: outcome_type, dtype: int64